# Least-to-most prompting

Complex problems often overwhelm language models when presented all at once. A sophisticated question requiring multiple reasoning steps, domain knowledge, and careful integration of information can exceed the model's ability to handle everything simultaneously. This is particularly true when problems have hierarchical structure - solving the overall problem depends on first solving simpler sub-problems.

Least-to-Most prompting addresses this by explicitly decomposing complex problems into a sequence of simpler sub-problems, solving them from easiest to hardest, and using each solution as context for the next. The technique consists of two stages: decomposition (break the problem into ordered sub-problems) and sequential solving (solve each sub-problem using previous solutions as context). This builds up understanding incrementally rather than requiring the model to handle all complexity at once.

In this notebook, we will implement Least-to-Most prompting to show how progressive problem decomposition improves reasoning on complex tasks. We will create a system that breaks down challenging questions into manageable pieces, solves them in order of increasing difficulty and builds each solution on previous ones. This approach is particularly effective for compositional reasoning, mathematical proofs, multi-hop question answering and any problem with natural hierarchical structure.

In [1]:
import os
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

We will use dataclasses to represent sub-problems and their solutions, maintaining clear structure as we build up from simple to complex.

### Initialize the language model
In Least-to-Most prompting, the language model serves dual purposes: decomposing problems into sub-problems and solving each sub-problem. We use a moderate temperature to allow flexibility in decomposition while maintaining logical consistency in solutions.

In [2]:
# Initialize the language model
# Using temperature=0.5 for balanced decomposition and solving
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY", "").strip(),
    temperature=0.5  # Moderate temperature for flexibility with consistency
)

Temperature 0.5 provides a good balance - creative enough to identify helpful decompositions, but consistent enough to produce reliable solutions to sub-problems.

## Define data structures
We need structures to represent sub-problems and track the solving process. Each sub-problem has a description and its solution. The solver maintains the progression from simple to complex, using earlier solutions as context for later ones.

In [3]:
@dataclass
class SubProblem:
    """
    Represents a single sub-problem in the decomposition.
    """
    index: int  # Order in the sequence (0 is simplest)
    description: str  # What this sub-problem asks
    solution: Optional[str] = None  # The answer (filled during solving)
    
    def is_solved(self) -> bool:
        """Check if this sub-problem has been solved."""
        return self.solution is not None
    
    def __repr__(self):
        """Pretty print representation."""
        status = "✓" if self.is_solved() else "○"
        return f"{status} Sub-problem {self.index + 1}: {self.description}"


@dataclass  
class DecompositionResult:
    """
    Result of decomposing a problem into sub-problems.
    """
    original_problem: str  # The original complex problem
    sub_problems: List[SubProblem]  # Ordered list from simplest to hardest
    
    def __repr__(self):
        """Pretty print the decomposition."""
        result = f"Original: {self.original_problem}\n\nSub-problems:\n"
        for sp in self.sub_problems:
            result += f"  {sp}\n"
        return result

- The `SubProblem` class represents individual sub-problems with their solutions.
- The `DecompositionResult` maintains the ordered sequence.

The ordering is crucial - we solve simpler problems first, using those solutions as building blocks for harder ones.

## Implement problem decomposition
The first stage of Least-to-Most is decomposition. We prompt the LLM to break the complex problem into simpler sub-problems, ordered from easiest to hardest. Good decomposition identifies natural sub-problems where each builds on previous ones.

In [4]:
def decompose_problem(problem: str, num_subproblems: int = 4) -> DecompositionResult:
    """
    Decompose a complex problem into simpler sub-problems.
    Orders them from least to most complex.
    
    Args:
        problem: The complex problem to decompose
        num_subproblems: Target number of sub-problems
    
    Returns:
        DecompositionResult with ordered sub-problems
    """
    # Create decomposition prompt
    system_message = SystemMessage(
        content=f"""You are an expert at breaking down complex problems into simpler sub-problems.
        
        Decompose the given problem into approximately {num_subproblems} sub-problems, ordered from SIMPLEST to MOST COMPLEX.
        Each sub-problem should build on the previous ones.
        
        Format your response as a numbered list:
        1. [First/simplest sub-problem]
        2. [Second sub-problem]
        3. [Third sub-problem]
        etc.
        
        Make each sub-problem clear and self-contained.
        """
    )
    
    user_message = HumanMessage(
        content=f"""Problem to decompose: {problem}
        
        Break this into simpler sub-problems:"""
    )
    
    # Get decomposition from LLM
    response = llm.invoke([system_message, user_message])
    decomposition_text = response.content
    
    # Parse the decomposition into SubProblem objects
    sub_problems = []
    lines = decomposition_text.split('\n')
    
    for line in lines:
        # Look for numbered lines (1., 2., etc.)
        line = line.strip()
        if line and line[0].isdigit():
            # Extract the sub-problem description
            # Remove the number prefix
            parts = line.split('.', 1)
            if len(parts) == 2:
                description = parts[1].strip()
                # Remove any markdown formatting
                description = description.strip('*').strip()
                if description:
                    sub_problems.append(
                        SubProblem(
                            index=len(sub_problems),
                            description=description
                        )
                    )
    
    return DecompositionResult(
        original_problem=problem,
        sub_problems=sub_problems
    )


# Test decomposition
test_problem = """A company wants to expand from 50 to 200 employees over 2 years.
They need to hire staff, find office space, and set up infrastructure.
What's a reasonable budget and timeline for this expansion?"""

decomposition = decompose_problem(test_problem)
print("=== Problem Decomposition ===")
print(decomposition)

=== Problem Decomposition ===
Original: A company wants to expand from 50 to 200 employees over 2 years.
They need to hire staff, find office space, and set up infrastructure.
What's a reasonable budget and timeline for this expansion?

Sub-problems:
  ○ Sub-problem 1: Determine the Staffing Needs**: Identify the specific roles and number of employees required for the expansion. This includes breaking down the types of positions needed (e.g., management, technical, administrative) and estimating the hiring timeline for each role.
  ○ Sub-problem 2: Estimate Hiring Costs**: Calculate the costs associated with the hiring process, including recruitment expenses (job postings, recruitment agency fees), salaries for the new employees, and any associated benefits. This should also include the costs for onboarding and training new hires.
  ○ Sub-problem 3: Identify Office Space Requirements**: Assess the space needed to accommodate the new employees. This involves determining the square foota

- The `decompose_problem()` function prompts the LLM to break down complexity into manageable pieces.
- The prompt explicitly requests ordering from simplest to most complex - this is key to Least-to-Most.
- We parse the response into structured `SubProblem` objects for systematic solving. Good decompositions have sub-problems where later ones naturally build on earlier answers.

## Implement sequential solving
The second stage solves each sub-problem in order, using previous solutions as context. This is the "most" part - we progressively tackle more complex sub-problems, leveraging what we have already solved. Each solution becomes part of the context for the next sub-problem.

In [5]:
def solve_subproblem(
    subproblem: SubProblem,
    previous_solutions: List[SubProblem],
    original_problem: str
) -> str:
    """
    Solve a single sub-problem using previous solutions as context.
    This is the core of Least-to-Most: building on previous answers.
    
    Args:
        subproblem: The sub-problem to solve
        previous_solutions: Already-solved sub-problems to use as context
        original_problem: The original complex problem (for context)
    
    Returns:
        Solution to this sub-problem
    """
    # Build context from previous solutions
    context = f"Original problem: {original_problem}\n\n"
    
    if previous_solutions:
        context += "Previous sub-problems and their solutions:\n"
        for prev in previous_solutions:
            context += f"\nQ{prev.index + 1}: {prev.description}\n"
            context += f"A{prev.index + 1}: {prev.solution}\n"
    else:
        context += "This is the first sub-problem.\n"
    
    # Create solving prompt
    system_message = SystemMessage(
        content="""You are solving sub-problems as part of a larger problem.
        Use the previous sub-problem solutions to help answer the current one.
        Be specific and build on what's already been established.
        """
    )
    
    user_message = HumanMessage(
        content=f"{context}\nCurrent sub-problem to solve:\n{subproblem.description}\n\nProvide a clear, specific answer:"
    )
    
    # Get solution from LLM
    response = llm.invoke([system_message, user_message])
    return response.content


def solve_sequentially(decomposition: DecompositionResult) -> List[SubProblem]:
    """
    Solve all sub-problems in order, each using previous solutions as context.
    
    Args:
        decomposition: The decomposed problem
    
    Returns:
        List of solved sub-problems
    """
    print("\n=== Sequential Solving ===")
    print(f"Solving {len(decomposition.sub_problems)} sub-problems from simplest to most complex...\n")
    
    solved_problems = []
    
    for subproblem in decomposition.sub_problems:
        print(f"[Sub-problem {subproblem.index + 1}/{len(decomposition.sub_problems)}]")
        print(f"Question: {subproblem.description}")
        
        # Solve using previous solutions as context
        solution = solve_subproblem(
            subproblem,
            solved_problems,  # Pass all previously solved sub-problems
            decomposition.original_problem
        )
        
        # Update the sub-problem with its solution
        subproblem.solution = solution
        solved_problems.append(subproblem)
        
        print(f"Answer: {solution[:150]}..." if len(solution) > 150 else f"Answer: {solution}")
        print()
    
    return solved_problems


# Test sequential solving
solved = solve_sequentially(decomposition)


=== Sequential Solving ===
Solving 4 sub-problems from simplest to most complex...

[Sub-problem 1/4]
Question: Determine the Staffing Needs**: Identify the specific roles and number of employees required for the expansion. This includes breaking down the types of positions needed (e.g., management, technical, administrative) and estimating the hiring timeline for each role.
Answer: To determine the staffing needs for the company's expansion from 50 to 200 employees, we can break down the roles into three main categories: manageme...

[Sub-problem 2/4]
Question: Estimate Hiring Costs**: Calculate the costs associated with the hiring process, including recruitment expenses (job postings, recruitment agency fees), salaries for the new employees, and any associated benefits. This should also include the costs for onboarding and training new hires.
Answer: To estimate the hiring costs associated with expanding the company from 50 to 200 employees, we will consider several components: recr

The `solve_subproblem()` function is where Least-to-Most shines. It provides all previous solutions as context, allowing the model to build incrementally. The `solve_sequentially()` function orchestrates this process, maintaining the progression from simple to complex. Each sub-problem benefits from what came before - this is how we handle complexity that would overwhelm the model if presented all at once.

## Synthesize final answer
After solving all sub-problems, we synthesize them into a comprehensive answer to the original question. This synthesis step combines the sub-problem solutions into a coherent whole, addressing the original complex problem.

In [6]:
def synthesize_final_answer(
    original_problem: str,
    solved_subproblems: List[SubProblem]
) -> str:
    """
    Synthesize a final comprehensive answer from all sub-problem solutions.
    
    Args:
        original_problem: The original complex problem
        solved_subproblems: All solved sub-problems
    
    Returns:
        Comprehensive final answer
    """
    # Build context from all solutions
    context = "Sub-problems and their solutions:\n\n"
    for sp in solved_subproblems:
        context += f"Q: {sp.description}\n"
        context += f"A: {sp.solution}\n\n"
    
    # Create synthesis prompt
    system_message = SystemMessage(
        content="""You are synthesizing a comprehensive answer from sub-problem solutions.
        Combine the insights from all sub-problems into a clear, complete answer
        to the original question. Make it coherent and well-structured.
        """
    )
    
    user_message = HumanMessage(
        content=f"{context}\nOriginal problem: {original_problem}\n\nProvide a comprehensive final answer:"
    )
    
    # Get final answer
    response = llm.invoke([system_message, user_message])
    return response.content


# Synthesize final answer
final_answer = synthesize_final_answer(test_problem, solved)

print("=== Final Comprehensive Answer ===")
print(final_answer)

=== Final Comprehensive Answer ===
To facilitate the expansion of the company from 50 to 200 employees over a two-year period, we have developed a comprehensive budget and timeline that addresses staffing needs, hiring costs, office space requirements, and infrastructure setup. Below is the detailed plan:

### Comprehensive Budget

1. **Total Hiring Costs:**
   - **Recruitment Expenses:**
     - Job Postings: $60,000
     - Recruitment Agency Fees: $2,100,000
     - **Total Recruitment Expenses:** **$2,160,000**
   
   - **Salaries:**
     - Management Salaries: $600,000
     - Technical Salaries: $1,680,000
     - Administrative Salaries: $1,000,000
     - **Total Salaries:** **$3,280,000**
   
   - **Benefits:**
     - Total Benefits Cost (30% of salaries): **$984,000**
   
   - **Onboarding and Training Costs:**
     - Total Onboarding and Training Costs: **$225,000**
   
   - **Overall Total Hiring Costs:** 
     - **Total Hiring Costs = Recruitment Expenses + Salaries + Benefits +

The `synthesize_final_answer()` function combines all sub-solutions into a coherent whole. It provides the full context of all sub-problems and solutions, asking the model to create a comprehensive answer. This synthesis step ensures we don't just have disconnected sub-answers, but an integrated response to the original complex question.

## Create complete Least-to-Most solver

Now we combine decomposition, sequential solving, and synthesis into a complete system that handles the full Least-to-Most workflow.

In [7]:
class LeastToMostSolver:
    """
    Complete Least-to-Most prompting solver.
    Decomposes problems, solves from simplest to most complex, synthesizes answer.
    """
    
    def solve(self, problem: str, verbose: bool = True) -> Dict[str, Any]:
        """
        Solve a complex problem using Least-to-Most prompting.
        
        Args:
            problem: The complex problem to solve
            verbose: Whether to print progress
        
        Returns:
            Dictionary with decomposition, solutions, and final answer
        """
        if verbose:
            print("=" * 80)
            print("LEAST-TO-MOST SOLVER")
            print("=" * 80)
            print(f"\nOriginal Problem: {problem}\n")
        
        # Stage 1: Decompose
        if verbose:
            print("[Stage 1: Decomposition]")
        decomposition = decompose_problem(problem)
        
        if verbose:
            print(f"Decomposed into {len(decomposition.sub_problems)} sub-problems:")
            for sp in decomposition.sub_problems:
                print(f"  {sp.index + 1}. {sp.description}")
        
        # Stage 2: Solve sequentially
        if verbose:
            print(f"\n[Stage 2: Sequential Solving]")
        solved = solve_sequentially(decomposition)
        
        # Stage 3: Synthesize
        if verbose:
            print("[Stage 3: Synthesis]")
        final_answer = synthesize_final_answer(problem, solved)
        
        if verbose:
            print("\n" + "=" * 80)
            print("FINAL ANSWER")
            print("=" * 80)
            print(final_answer)
        
        return {
            'problem': problem,
            'decomposition': decomposition,
            'solved_subproblems': solved,
            'final_answer': final_answer
        }


# Test the complete solver
solver = LeastToMostSolver()
result = solver.solve(
    "How can a small startup with 5 people and $50k budget launch a mobile app and get 10,000 users in 6 months?"
)

LEAST-TO-MOST SOLVER

Original Problem: How can a small startup with 5 people and $50k budget launch a mobile app and get 10,000 users in 6 months?

[Stage 1: Decomposition]
Decomposed into 4 sub-problems:
  1. Define the App Concept and Target Audience**: Identify the core features and functionalities of the mobile app, as well as the specific target audience. This includes conducting market research to understand user needs and preferences.
  2. Develop a Minimum Viable Product (MVP)**: Create a basic version of the app that includes only the essential features necessary to meet the needs of the target audience. This involves planning the app's design, user interface, and technical requirements, and then allocating resources for development.
  3. Establish a Marketing Strategy**: Formulate a marketing plan to promote the app and attract users. This includes selecting appropriate marketing channels (social media, content marketing, etc.), setting a budget for advertising, and creating

The `LeastToMostSolver` class orchestrates the complete three-stage process: decompose the problem into sub-problems ordered by difficulty, solve each sequentially using previous solutions as context, and synthesize a comprehensive final answer. This structured approach handles complex problems that would overwhelm direct solving.

## Compare with direct solving
To appreciate Least-to-Most's value, let's compare it with asking the LLM to solve complex problems directly. Direct solving requires handling all complexity simultaneously, while Least-to-Most builds understanding incrementally.

In [8]:
def solve_directly(problem: str) -> str:
    """
    Solve problem directly without decomposition.
    """
    response = llm.invoke([HumanMessage(content=problem)])
    return response.content


# Compare approaches
comparison_problem = "Design a complete employee onboarding system for a remote-first company that integrates training, equipment setup, team introductions, and compliance documentation."

print("=" * 80)
print("COMPARISON: Least-to-Most vs Direct Solving")
print("=" * 80)

print("\n[Method 1: Direct Solving]")
print("-" * 40)
direct = solve_directly(comparison_problem)
print(direct)

print("\n[Method 2: Least-to-Most]")
print("-" * 40)
ltm_result = solver.solve(comparison_problem, verbose=False)
print(ltm_result['final_answer'])

COMPARISON: Least-to-Most vs Direct Solving

[Method 1: Direct Solving]
----------------------------------------
Designing a complete employee onboarding system for a remote-first company involves creating a structured process that ensures new hires feel welcomed, informed, and equipped to succeed in their roles. Below is a comprehensive outline for such a system, integrating training, equipment setup, team introductions, and compliance documentation.

### Employee Onboarding System

#### 1. Pre-Onboarding Phase (1-2 weeks before start date)

**1.1 Welcome Email**
- Send a personalized welcome email that includes:
  - Start date and time
  - Overview of the onboarding process
  - Introduction to the company culture and values
  - Links to the company website and social media

**1.2 Onboarding Portal Access**
- Grant access to an onboarding portal where new hires can:
  - Review the onboarding schedule
  - Access necessary documents and resources
  - Complete pre-employment forms

**1.3

This comparison reveals Least-to-Most's systematic approach. While direct solving might produce reasonable answers quickly, Least-to-Most ensures comprehensive coverage by explicitly breaking down and addressing each aspect and provides transparent reasoning process. The decomposition forces systematic thinking that direct solving might miss.

Least-to-Most prompting addresses complex problems through progressive decomposition and sequential solving. By breaking problems into simpler sub-problems and solving them in order of increasing difficulty, we handle complexity that would overwhelm all-at-once approaches.

**When to use Least-to-Most:**
- Complex problems with natural sub-problems.
- Hierarchical reasoning tasks.
- Problems where later steps depend on earlier ones.
- Compositional questions.
- Multi-step proofs or derivations.

**Implementation tips:**
- Order sub-problems from simplest to most complex.
- Ensure each sub-problem builds on previous ones.
- Provide all previous solutions as context.
- Synthesize sub-solutions into coherent whole.

**Comparison with alternatives:**
- **vs direct solving**: More systematic but slower.
- **vs chain-of-thought**: More structured decomposition.
- **vs tree-of-thoughts**: Linear progression vs exploration.

Least-to-Most demonstrates that tackling complexity incrementally - from least to most difficult - often produces better results than attacking everything simultaneously.